In [46]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [47]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')
tables = soup.find_all('table',class_='wikitable sortable')

In [3]:
len(tables)

4

In [54]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [55]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
df_2022 = df[['Title','Cast and crew']]

In [56]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'a6a1a53e9ce9c20e950add9d7b94b480'

In [57]:
df_2022.head()


,Title,Cast and crew
0,The 355,Simon Kinberg (director/screenplay); Theresa R...
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...
3,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director..."
4,Hotel Transylvania: Transformania,"Jennifer Kluska, Derek Drymon (directors); Amo..."


In [58]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
result = tmdb_movie.search(df_2022.iloc[1,0])

In [59]:
movie_id = result[0].id
response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
data_json = response.json()

In [60]:
data_json
    


{'adult': False,
 'backdrop_path': '/9BQPM1Fa1hdXaE7hNafO5WWNv9v.jpg',
 'belongs_to_collection': None,
 'budget': 0,
 'genres': [{'id': 27, 'name': 'Horror'}, {'id': 53, 'name': 'Thriller'}],
 'homepage': '',
 'id': 631947,
 'imdb_id': 'tt7267498',
 'original_language': 'en',
 'original_title': 'La Llorona',
 'overview': "While vacationing in Mexico, a couple discovers their son's disappearance is tied to a supernatural curse.",
 'popularity': 45.827,
 'poster_path': '/wtgEJKEOLnyQkWemEdFa5W8Q29L.jpg',
 'production_companies': [{'id': 76992,
   'logo_path': '/mw3j3P8SUQJKLSJQrLKuSDO9Xr6.png',
   'name': 'Saban Films',
   'origin_country': 'US'}],
 'production_countries': [{'iso_3166_1': 'MX', 'name': 'Mexico'}],
 'release_date': '2022-01-07',
 'revenue': 0,
 'runtime': 98,
 'spoken_languages': [{'english_name': 'English',
   'iso_639_1': 'en',
   'name': 'English'},
  {'english_name': 'Spanish', 'iso_639_1': 'es', 'name': 'Español'}],
 'status': 'Released',
 'tagline': 'Her spirit live

In [61]:
data_json['overview']

"While vacationing in Mexico, a couple discovers their son's disappearance is tied to a supernatural curse."

In [62]:
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

def getreleasedate(x):
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
    if data_json['release_date']:
          return data_json['release_date']
    else:
        return np.nan


def getruntime(x):
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
    if data_json['runtime']:
          return data_json['runtime']
    else:
        return np.nan

def getoverview(x):
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
    if data_json['overview']:
          return data_json['overview']
    else:
        return np.nan





df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))
df_2022['overview'] = df_2022['Title'].map(lambda x: getoverview(str(x)))
df_2022['releasedate'] = df_2022['Title'].map(lambda x: getreleasedate(str(x)))
df_2022['duration'] = df_2022['Title'].map(lambda x: getruntime(str(x)))

def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))

def getruntime(x):
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
    if data_json['runtime']:
          return data_json['runtime']
    else:
        return np.nan
    
df_2022['duration'] = df_2022['Title'].map(lambda x: getruntime(str(x)))

    

/var/folders/s8/ry4t0lwd27d51g6ly2d7zhtr0000gn/T/ipykernel_21553/1063641196.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))
/var/folders/s8/ry4t0lwd27d51g6ly2d7zhtr0000gn/T/ipykernel_21553/1063641196.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022['overview'] = df_2022['Title'].map(lambda x: getoverview(str(x)))
/var/folders/s8/ry4t0lwd27d51g6ly2d7zhtr0000gn/T/ipykernel_21553/1063641196.py:64: SettingWithCopyWarning: 
A 

In [33]:
data = pd.read_csv('../data.csv')
data.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action|Adventure|Fantasy|Sci-Fi,avatar
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action|Adventure|Fantasy,pirates of the caribbean: at world's end
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action|Adventure|Thriller,spectre
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action|Thriller,the dark knight rises
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...


In [34]:
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

def getreleasedate(x):
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
    if data_json['release_date']:
          return data_json['release_date']
    else:
        return np.nan


def getruntime(x):
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
    if data_json['runtime']:
          return data_json['runtime']
    else:
        return np.nan

def getoverview(x):
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
    if data_json['overview']:
          return data_json['overview']
    else:
        return np.nan



data['overview'] = data['movie_title'].map(lambda x: getoverview(str(x)))
data['releasedate'] = data['movie_title'].map(lambda x: getreleasedate(str(x)))
data['duration'] = data['movie_title'].map(lambda x: getruntime(str(x)))




    

In [45]:
df_2022.columns

Index(['Title', 'Cast and crew', 'genres', 'overview', 'releasedate',
       'director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name'],
      dtype='object')

In [43]:
data.columns

Index(['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name',
       'genres', 'movie_title', 'overview', 'releasedate', 'duration'],
      dtype='object')

In [67]:
data.columns
data.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,overview,releasedate,duration
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action|Adventure|Fantasy|Sci-Fi,avatar,"In the 22nd century, a paraplegic Marine is di...",2009-12-10,162.0
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action|Adventure|Fantasy,pirates of the caribbean: at world's end,"Captain Barbossa, long believed to be dead, ha...",2007-05-19,169.0
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action|Adventure|Thriller,spectre,A cryptic message from Bond’s past sends him o...,2015-10-26,148.0
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action|Thriller,the dark knight rises,Following the death of District Attorney Harve...,2012-07-16,165.0
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Thirty years after defeating the Galactic Empi...,2015-12-15,136.0


In [69]:
df_2022.tail()


,Title,Cast and crew,genres,overview,releasedate,duration,director_name,actor_1_name,actor_2_name,actor_3_name
166,Puss in Boots: The Last Wish,"Joel Crawford (director); Antonio Banderas, Sa...",Animation Adventure Comedy Family Fantasy Drama,Puss in Boots discovers that his passion for a...,2022-12-21,NaN,Joel Crawford,Joel Crawford (director); Antonio Banderas,Salma Hayek,Harvey Guillén
167,I Wanna Dance with Somebody,Kasi Lemmons (director); Anthony McCarten (scr...,Drama,Biopic of the legendary singer Whitney Houston.,2022-12-21,NaN,Kasi Lemmons,Naomi Ackie,Ashton Sanders,Stanley Tucci
168,Babylon,Damien Chazelle (director/screenplay); Brad Pi...,Action Adventure Science Fiction Thriller,A veteran-turned-mercenary is hired to take a ...,2008-08-20,101.0,Damien Chazelle,Brad Pitt,Margot Robbie,Tobey Maguire
169,A Man Called Otto,"Marc Forster (director), David Magee (screenpl...",Drama Comedy,"Otto, a grumpy and strict widower, holds the r...",2022-12-25,NaN,Marc Forster,Tom Hanks,NaN,NaN
170,NaN,NaN,Comedy,Catherine Tate's iconic character Nan hits the...,2022-03-18,95.0,nan,nan,NaN,NaN
